In [70]:
include("StackyFan.jl")

coneRayDecomposition (generic function with 1 method)

In [71]:
#inputs are a stacky fan and a vector of booleans representing the distinguished structure.
function BerghA(F::StackyFan,D::Array{Int64,1})
    X=deepcopy(F)
    rayMatrix=convert(Array{Int64,2},Array(Polymake.common.primitive(X.fan.RAYS)))
    coneList=convertIncidenceMatrix(X.fan.MAXIMAL_CONES)
    dim=size(rayMatrix,2)
    l=size(rayMatrix,1)
    #check if the vector D has length equal to the number of rays in F
    if l != size(D,1)
        error("length of vector representing distinguished structure does not agree with number of rays in stacky fan.")
    end
    
    #A0
    while(true)
        #A1
        #Note: cones in S are 1-indexed.
        S=filter(cone->distinguishedAndMultiplicity(cone,rayMatrix,D),coneList)
        if S==[]
            break
        end
        
        #A2 - find extremal cones
        Smax=extremalCones(S,rayMatrix,D)
        
        #A2 - find interior points in Smax
        intPoints=[]
        for cone in Smax
            C=coneConvert(cone,rayMatrix)
            coneIntPoints=interiorPoints(C)
            for point in coneIntPoints
               push!(intPoints,(point,cone)) 
            end
        end
        
        #A2 - find stacky points (in terms of coefficients) derived from interior points
        P=Array{Int64,1}[]
        for (point,cone) in intPoints
            stackyPoint=coneRayDecomposition(cone,rayMatrix,point,X.scalars)
            push!(P,stackyPoint)
        end
        
        #A2 - find smallest element of P with respect to lex ordering.
        print(P)
        psi=minimalByLex(P)

        #A3 - perform root construction
        X=rootConstructionDistinguishedIndices(X,D,psi)
        rayMatrix=convert(Array{Int64,2},Array(Polymake.common.primitive(X.fan.RAYS)))
        
        #A3 - modify psi with respect to root construction
        for i in 1:length(psi)
            if D[i]==1
                psi[i]=1
            end
        end
        
        while(count(x->x>0,psi)>1)
            #A4 - perform stacky star subdivision
            supportCone=findall(x->x==1,psi)
            blowupRay=rayMatrix*psi
            X=stackyBlowup(X,supportCone,blowupRay)
        
            #A4 - modify psi
            newpsi=zeros(Int64,length(psi)+1)
            for i in 1:length(psi)
                if psi[i]>1
                    newpsi[i]=psi[i]-1
                end
            end
            newpsi[end]=1
        end
    end
    return X
end

BerghA (generic function with 1 method)

In [72]:
X=Polymake.fulton.NormalToricVariety(INPUT_RAYS=[1 2;1 0],INPUT_CONES=[[0,1]])

type: NormalToricVariety

INPUT_CONES
	{0 1}

INPUT_RAYS
	1 2
	1 0

In [73]:
F=addStackStructure(X,[1,1])

StackyFan(Polymake.BigObjectAllocated(Ptr{Nothing} @0x0000000019f0b220), [1, 1], Dict("1,2" => 1,"1,0" => 1))

In [74]:
BerghA(F,[1,1])

[[1, 1]]

┌ Warning: Interrupting polymake is not safe.
│ SIGINT is disabled while waiting for polymake to finish its computations.
└ @ Polymake /home/kitty/.julia/packages/Polymake/qeOTA/src/perlobj.jl:47


LoadError: [91mInterruptException:[39m

In [4]:
A=[0,1,2,3]
A[end]

3